In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [87]:
val topic_name = "lab04_input_data"
val offset = "earliest"
val output_dir_prefix = "/user/andrey.blednykh2/visits"

topic_name = lab04_input_data
offset = earliest
output_dir_prefix = /user/andrey.blednykh2/visits


/user/andrey.blednykh2/visits

In [88]:
val kafkaParams = Map(
        "kafka.bootstrap.servers" -> "spark-master-1:6667",
        "subscribe" -> topic_name,
        "startingOffsets" -> offset
    )

kafkaParams = Map(kafka.bootstrap.servers -> spark-master-1:6667, subscribe -> lab04_input_data, startingOffsets -> earliest)


Map(kafka.bootstrap.servers -> spark-master-1:6667, subscribe -> lab04_input_data, startingOffsets -> earliest)

In [89]:
val sdf_kafka = spark
    .read
    .format("kafka")
    .options(kafkaParams)
    .load

sdf_kafka = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [90]:
val parsed_sdf_kafka = sdf_kafka
    .select('key, 'value.cast("string"), 'topic, 'partition, 'offset, 'timestamp, 'timestampType)

parsed_sdf_kafka = [key: binary, value: string ... 5 more fields]


[key: binary, value: string ... 5 more fields]

In [91]:
val schema = StructType(Seq(
    StructField("event_type", StringType, true),    
    StructField("category", StringType, true),
    StructField("item_id", StringType, true),
    StructField("item_price", IntegerType, true),
    StructField("uid", StringType, true),
    //StructField("timestamp", StringType, true)
    //StructField("timestamp", TimestampType, true)
    StructField("timestamp", LongType, true)
))

schema = StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,IntegerType,true), StructField(uid,StringType,true), StructField(timestamp,LongType,true))


StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,IntegerType,true), StructField(uid,StringType,true), StructField(timestamp,LongType,true))

In [92]:
val value_sdf_kafka = parsed_sdf_kafka.select('value.cast("string"))

value_sdf_kafka = [value: string]


[value: string]

In [93]:
val json_sdf_kafka = value_sdf_kafka
    .withColumn("value", from_json($"value", schema))
    .select($"value.*")

json_sdf_kafka = [event_type: string, category: string ... 4 more fields]


[event_type: string, category: string ... 4 more fields]

In [94]:
json_sdf_kafka.show(1,200,true)

-RECORD 0------------------------------------------
 event_type | buy                                  
 category   | Entertainment-equipment              
 item_id    | Entertainment-equipment-2            
 item_price | 2529                                 
 uid        | 40b29579-e845-45c0-a34d-03630d296a81 
 timestamp  | 1577865600000                        
only showing top 1 row



In [95]:
val k_date = json_sdf_kafka
    .withColumn("date", 
                trim(date_format(to_timestamp((col("timestamp")/1000).cast("long")),"yyyyMMdd")))
    .withColumn("p_date", 
                trim(date_format(to_timestamp((col("timestamp")/1000).cast("long")),"yyyyMMdd")))
    .cache()

k_date = [event_type: string, category: string ... 6 more fields]


[event_type: string, category: string ... 6 more fields]

In [96]:
val k_view_date = k_date
    .select("category","event_type","item_id","item_price","timestamp","uid","date","p_date")
    .filter("event_type = 'view'")

k_view_date = [category: string, event_type: string ... 6 more fields]


[category: string, event_type: string ... 6 more fields]

In [97]:
val k_buy_date = k_date
    .select("category","event_type","item_id","item_price","timestamp","uid","date","p_date")
    .filter("event_type = 'buy'")

k_buy_date = [category: string, event_type: string ... 6 more fields]


[category: string, event_type: string ... 6 more fields]

In [98]:
//import spark.implicits._
k_view_date
    .repartition($"p_date")
    .write
    .partitionBy("p_date")
    .json(s"$output_dir_prefix/view/")

In [100]:
//import spark.implicits._
k_buy_date
    .repartition($"p_date")
    .write
    .partitionBy("p_date")
    .json(s"$output_dir_prefix/buy/")